In [2]:
import numpy as np
def sigmoid(z):
    """
    Compute the sigmoid of z

    Arguments:
    z -- A scalar or numpy array of any size.

    Return:
    s -- sigmoid(z)
    """

    s = 1/(1+np.exp(-z))
    
    return s

def initialize_with_zeros(dim):
    """
    This function creates a vector of zeros of shape (dim, 1) for w and initializes b to 0.
    
    Argument:
    dim -- size of the w vector we want (or number of parameters in this case)
    
    Returns:
    w -- initialized vector of shape (dim, 1)
    b -- initialized scalar (corresponds to the bias)
    """
    
    w = np.zeros((dim,1))
    b = 0.

    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

def propagate(w, b, X, Y):
    """
    Implement the cost function and its gradient for the propagation explained above

    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    Y -- true "label" vector of size (1, number of examples)

    Return:
    cost -- negative log-likelihood cost for logistic regression
    dw -- gradient of the loss with respect to w, thus same shape as w
    db -- gradient of the loss with respect to b, thus same shape as b
    
    Tips:
    - Write your code step by step for the propagation. np.log(), np.dot()
    """
    
    m = np.float(X.shape[1])
    
    # FORWARD PROPAGATION (FROM X TO COST)
    A = sigmoid(np.dot(w.T,X) + b)                                    # compute activation
    cost = -(1/m) * (np.sum((Y*np.log(A))+ ((1-Y)*np.log(1-A))))     # compute cost
    
    # BACKWARD PROPAGATION (TO FIND GRAD)
    dw = (1/m)*(np.dot(X,(A-Y).T))
    db = (1/m)*(np.sum(A-Y))

    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    """
    This function optimizes w and b by running a gradient descent algorithm
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector, of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b.
    """
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation (≈ 1-4 lines of code)
        ### START CODE HERE ### 
        grads, cost = propagate(w,b,X,Y)
        ### END CODE HERE ###
        
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule (≈ 2 lines of code)
        ### START CODE HERE ###
        w = w - learning_rate*dw
        b = b - learning_rate*db
        ### END CODE HERE ###
        
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

def predict(w, b, X):
    '''
    Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
    
    Arguments:
    w -- weights, a numpy array of size (num_px * num_px * 3, 1)
    b -- bias, a scalar
    X -- data of size (num_px * num_px * 3, number of examples)
    
    Returns:
    Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
    '''
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m), dtype=np.int)
    w = w.reshape(X.shape[0], 1)
    
    # Compute vector "A" predicting the probabilities of a news being present
    A = sigmoid(np.dot(w.T,X) + b)
    
    for i in range(A.shape[1]):
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        if (A[0][i] > 0.5):
            Y_prediction[0][i] = 1
        else:
            Y_prediction[0][i] = 0
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction, A

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    """
    Builds the logistic regression model by calling the function you've implemented previously
    
    Arguments:
    X_train -- training set represented by a numpy array of shape (num_px * num_px * 3, m_train)
    Y_train -- training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test -- test set represented by a numpy array of shape (num_px * num_px * 3, m_test)
    Y_test -- test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations -- hyperparameter representing the number of iterations to optimize the parameters
    learning_rate -- hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost -- Set to true to print the cost every 100 iterations
    
    Returns:
    d -- dictionary containing information about the model.
    """

    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent (≈ 1 line of code)
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test, sigmoid_result_test = predict(w,b,X_test)
    Y_prediction_train, sigmoid_result_train = predict(w,b,X_train)
    
    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train,
         "sigmoid_result_test": sigmoid_result_test,
         "sigmoid_result_train": sigmoid_result_train,
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [10]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pandas
from sklearn import model_selection, preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
import logisticregression as lgr
import copy
from pprint import pprint

np.random.seed(0)
dfs = []
dfs.append(pandas.read_csv('ThoiSu.csv'))
dfs.append(pandas.read_csv('TheThao.csv'))
dfs.append(pandas.read_csv('TheGioi.csv'))
df = pandas.concat(dfs, ignore_index=True)

###### 1. Thu thập một số bài báo từ các thể loại khác nhau
# đếm số dòng ThoiSu
size_1 = df[ df['label'] == 'ThoiSu' ].shape[0]
# đếm số dòng TheThao
size_2 = df[ df['label'] == 'TheThao' ].shape[0]
# đếm số dòng TheGioi
size_3 = df[ df['label'] == 'TheGioi' ].shape[0]

print('Số tài liệu ThoiSu: %s' %size_1)
print('Số tài liệu TheThao: %s' %size_2)
print('Số tài liệu TheGioi: %s' %size_3)

Số tài liệu ThoiSu: 2895
Số tài liệu TheThao: 2918
Số tài liệu TheGioi: 5650


In [8]:
# tách dữ liệu thành 2 tập training và test: train va test
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['text'], df['label'])

# đưa label ve 0 va 1
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

# print('chuyển label ["ThoiSu", "TheThao", "TheGioi"] thành %s' %encoder.transform(["ThoiSu", "TheThao", "TheGioi"]))

###### 2. Thu thập một số bài báo từ các thể loại khác nhau
# word level tf-idf
tfidf_vect = TfidfVectorizer(encoding='utf-16', analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df['text'])

xtrain_tfidf =  tfidf_vect.transform(train_x)
xtest_tfidf =  tfidf_vect.transform(test_x)

print('Số tài liệu training: %s' %str(xtrain_tfidf.shape[0]))
print('Số tài liệu testing: %s' %str(xtest_tfidf.shape[0]))
print('Number of features of each document: %s' %str(xtrain_tfidf.shape[1]))

# Many software bugs in deep learning come from having matrix/vector dimensions that don't fit. If you can keep your matrix/vector dimensions straight you will go a long way toward eliminating many bugs.
# Exercise:: For convenience, you should now transpose the training and testing np-array, and expand the shape of the lable arrays in the axis=0 position.
# After this, our training (and test) dataset is a np-array where each column represents a document vector. There should be the number of training documents (respectively the number of testing documents) as the number of columns.

train_y = np.expand_dims(train_y, axis=0)
test_y = np.expand_dims(test_y, axis=0)
# print('===============')
# convert sparse to dense matrix 
xtrain_tfidf =  xtrain_tfidf.T.toarray() 
xtest_tfidf =  xtest_tfidf.T.toarray()

Số tài liệu training: 8597
Số tài liệu testing: 2866
Number of features of each document: 5000


In [6]:
dict = tfidf_vect.vocabulary_ # Shape {word : index}
pprint(xtrain_tfidf.shape[1])

for i in range(xtrain_tfidf.shape[1]):

    label = encoder.inverse_transform([train_y[0, i]])
    priorityQueue = []

    for word in dict:

        score = xtrain_tfidf[dict[word], i]
        heapq.heappush(priorityQueue, (-score, word))

    top10 = heapq.nsmallest(10, priorityQueue)
    wordList = [top10[i][1] for i in range(10)]
    print(i, wordList, label)

2854
0 ['nhị', 'thọ', 'bay', 'xuân', 'hung', 'thanh', 'sân', 'lê', 'viên', 'hoá'] ['ThoiSu']
1 ['hồ', 'nước', 'thủy', 'lợi', 'phú', 'lòng', 'cạn', 'ninh', 'tích', 'mưa'] ['TheGioi']
2 ['izumo', 'js', 'nhật', 'hoán', 'tàu', 'bay', 'tokyo', 'cải', 'sân', 'bản'] ['TheGioi']
3 ['di', 'cư', 'lhq', 'quốc', 'thỏa', 'hiệp', 'ước', 'gia', 'những', 'thuận'] ['TheGioi']
4 ['chatbot', 'messenger', '1022', 'tin', 'đài', 'cập', 'truy', 'nhắn', 'du', 'thông'] ['TheThao']
5 ['bão', 'thạnh', 'tán', 'tránh', 'cần', 'sơ', 'dân', 'bà', 'usagi', 'giờ'] ['TheGioi']
6 ['huyện', 'thi', 'chấm', 'giáo', 'thí', 'cáo', 'sơn', 'đơn', 'viên', 'tuyển'] ['TheThao']
7 ['tháo', 'đạn', 'bom', 'ông', 'hưng', 'nghề', 'phế', 'thuốc', 'gio', 'nổ'] ['TheThao']
8 ['dubai', 'nhảy', 'bà', 'dù', '82', 'mơ', 'ước', 'mét', 'cùng', 'cụ'] ['TheGioi']
9 ['tuyền', 'lâm', 'hồ', 'gỗ', 'tích', 'căn', 'dãy', 'dưỡng', 'xâm', 'di'] ['ThoiSu']
10 ['xây', 'dựng', 'sóc', 'vi', 'phạm', 'đất', 'xử', 'rừng', 'sơn', 'nông'] ['TheGioi']
11 ['canada

124 ['ghe', 'con', 'bơi', 'lật', 'gái', 'vợ', 'chèo', 'ngãi', 'cuốn', 'nhưng'] ['TheGioi']
125 ['tỉnh', 'ubnd', 'quảng', 'tịch', 'thuật', 'bí', 'thư', 'chủ', 'bầu', '57'] ['TheGioi']
126 ['ôtô', 'xe', 'tông', 'cẩm', 'tóe', 'tải', 'tĩnh', 'chở', 'tung', 'chạy'] ['ThoiSu']
127 ['cỏ', 'công', 'mọc', 'tùm', 'um', 'trình', 'khu', 'diễn', 'tỷ', 'vốn'] ['TheGioi']
128 ['golfer', 'gậy', 'giang', 'hữu', 'bảng', 'đấu', '6d', 'masters', 'truất', 'điểm'] ['TheThao']
129 ['thiêm', 'ranh', 'phố', 'khu', 'hoạch', 'phường', 'quy', 'thủ', 'hộ', 'khiếu'] ['ThoiSu']
130 ['ha', 'cư', 'tái', 'khu', 'hoạch', 'quy', 'thiêm', 'định', 'ranh', 'đất'] ['TheThao']
131 ['tàu', 'ôtô', 'húc', 'văng', 'vong', 'nữ', 'tử', 'hỏa', 'phụ', 'băng'] ['TheThao']
132 ['campuchia', 'việt', 'asean', 'và', 'coc', 'nam', 'pháp', 'mekong', 'cư', 'tướng'] ['TheGioi']
133 ['vec', 'tốc', 'ngãi', 'lún', 'nẵng', 'quảng', 'đà', 'cao', 'thầu', 'cầu'] ['ThoiSu']
134 ['rác', 'nghệ', 'đốt', 'chôn', 'công', 'loại', 'xử', 'lý', 'ta', 'lấp'] [

220 ['xe', 'kẹt', 'đường', 'bến', 'ngập', 'trễ', 'miền', 'gòn', 'xô', 'sài'] ['TheThao']
221 ['người', 'việt', 'gốc', 'mỹ', 'cộng', 'trục', 'những', 'biểu', 'california', 'đảng'] ['TheGioi']
222 ['rừng', 'tra', 'sóc', 'đất', 'thanh', 'xã', 'lý', 'minh', 'sơn', 'hà'] ['ThoiSu']
223 ['ngập', 'quận', 'bay', 'đường', 'xe', 'mưa', 'máy', 'tân', 'chuyến', 'bình'] ['TheGioi']
224 ['bay', 'vietjet', 'thuột', 'bánh', 'ma', 'buôn', 'máy', 'hàng', 'cánh', 'sân'] ['TheThao']
225 ['rôn', 'băng', 'cờ', 'sản', 'xưởng', 'bóng', 'nhung', 'sticker', 'buôn', 'vân'] ['TheThao']
226 ['hồ', 'nước', 'thủy', 'lợi', 'phú', 'lòng', 'cạn', 'ninh', 'tích', 'mưa'] ['ThoiSu']
227 ['lạng', 'linh', 'sư', 'tòa', 'vật', 'tử', 'lai', 'cửa', 'đá', 'ngoại'] ['TheThao']
228 ['voọc', 'ngãi', 'tốc', 'quảng', 'kiểm', 'lâm', 'iucn', 'nẵng', 'vá', 'nvcc'] ['TheThao']
229 ['đê', 'mã', 'kè', 'sạt', 'rồng', 'sông', 'hàm', 'nứt', 'lở', 'bê'] ['TheGioi']
230 ['ngựa', 'đua', 'trường', 'dự', 'xã', 'tân', 'nội', 'án', 'hà', 'minh'] ['T

319 ['afghanistan', 'taliban', 'mỹ', 'quân', 'phiến', 'mìn', 'binh', 'mạng', 'thiệt', 'almasdar'] ['TheGioi']
320 ['ngập', 'nẵng', 'đà', 'mưa', 'đường', 'cm', 'vỉa', 'thám', 'xe', 'hè'] ['TheGioi']
321 ['biểu', 'tình', 'elysees', 'champs', 'strasbourg', 'ngực', 'vàng', 'áo', 'tuần', 'người'] ['TheGioi']
322 ['giáo', 'áp', 'cô', 'nhạ', 'thầy', 'lực', 'sinh', 'học', 'dục', 'đua'] ['ThoiSu']
323 ['nhũng', 'checchi', 'francesco', 'tham', 'giàu', 'unodc', 'ông', 'quan', 'kim', 'sản'] ['TheGioi']
324 ['bắc', 'rét', 'độ', 'miền', 'mùa', 'hà', 'gió', 'c', 'nhiệt', 'đông'] ['ThoiSu']
325 ['bé', 'bèo', 'sông', 'tuấn', 'tuổi', 'anh', 'vớt', 'kiên', 'may', 'rơi'] ['ThoiSu']
326 ['cá', 'chết', 'hồ', 'phường', 'thối', 'cửa', 'vinh', 'kín', 'rô', 'hôi'] ['TheThao']
327 ['nga', 'putin', 'ước', 'mơ', 'trở', 'thống', 'tổng', 'đặc', 'phỏng', 'biệt'] ['TheGioi']
328 ['paris', 'tình', 'biểu', 'bạo', 'sát', 'trạng', 'pháp', 'trào', 'đối', 'cảnh'] ['TheGioi']
329 ['jetstar', 'bốc', 'xếp', 'bay', 'hóa', 'paci

432 ['lexus', 'xe', 'đâm', 'cồn', 'nồng', 'tai', 'nạn', 'miligam', 'thở', 'hà'] ['TheThao']
433 ['hđnd', 'tín', 'phố', 'nhiệm', 'phiếu', 'thành', 'họp', 'năm', 'bổ', 'chốt'] ['TheThao']
434 ['đua', 'tổ', 'f1', 'giải', 'chức', 'thức', '2020', 'hà', 'á', 'gala'] ['TheGioi']
435 ['lexus', 'xe', 'vỉa', 'hè', 'va', 'xế', 'chiếc', 'chạm', 'lao', 'chỗ'] ['TheGioi']
436 ['học', 'phí', 'thcs', 'giảm', 'hcm', '000', 'miễn', 'tp', 'hđnd', 'đồng'] ['TheThao']
437 ['khách', 'bay', 'chiếu', 'tân', 'bokang', 'masena', 'airasia', 'jon', 'sơn', 'nhật'] ['ThoiSu']
438 ['hđnd', 'tỉnh', 'tín', 'phiếu', 'hoá', 'biểu', 'nhiệm', 'chiến', 'sẽ', 'bí'] ['TheGioi']
439 ['bão', 'usagi', 'nam', 'vũng', 'rịa', '23', 'cấp', 'thuận', 'mưa', 'km'] ['ThoiSu']
440 ['onyx', 'lockheed', 'binh', 'xương', 'khung', 'martin', 'mỹ', 'lính', 'sĩ', 'chiến'] ['TheGioi']
441 ['lồng', 'lũ', 'cá', 'trôi', '40', 'cuốn', 'mưa', 'ngọc', 'nuôi', 'ôxy'] ['TheThao']
442 ['thạch', 'tĩnh', 'hà', 'đường', 'nhánh', 'km', 'cụt', 'tuyến', 'tỉnh

...

2435 ['màn', 'trận', 'lượt', 'xem', 'tổ', 'lắp', 'aff', 'asiad', 'cup', 'cổ'] ['ThoiSu']
2436 ['nang', 'mực', 'ngư', 'tàu', 'bến', 'tấn', '250', 'thu', 'mai', 'pv'] ['ThoiSu']
2437 ['vượt', 'm', 'nút', 'cầu', 'tỷ', '2012', 'câ', 'u', 'hà', 'đầu'] ['ThoiSu']
2438 ['lav', 'giáp', 'tàu', 'tiêu', 'lục', 'chiến', 'thiết', 'mục', 'binh', 'uss'] ['TheGioi']
2439 ['abdullah', 'visa', 'mỹ', 'bé', 'con', 'hassan', 'yemen', 'gặp', 'mẹ', 'bệnh'] ['TheGioi']
2440 ['daniels', 'trump', 'stormy', 'dâm', 'khiêu', 'phán', 'cô', 'bác', 'kiện', 'sư'] ['TheGioi']
2441 ['thang', 'chiếc', 'hãi', 'ấn', 'sợ', 'hồn', 'làng', 'nó', 'độ', 'bề'] ['TheGioi']
2442 ['chăn', 'nông', 'trưởng', 'phùng', 'tiến', 'nuôi', 'nghiệp', 'thứ', 'phó', 'đức'] ['TheThao']
2443 ['xế', 'tài', 'chu', 'tôn', 'khách', 'cướp', 'tệ', 'giết', 'hành', 'tiền'] ['TheGioi']
2444 ['cẩm', 'ốt', 'ki', 'ôtô', 'nạn', 'xuyên', 'tai', 'tông', 'đường', 'máy'] ['TheGioi']
2445 ['golfer', 'gậy', 'giang', 'hữu', 'bảng', 'đấu', '6d', 'masters', 'truất', 

2528 ['ngựa', 'đua', 'trường', 'dự', 'xã', 'tân', 'nội', 'án', 'hà', 'minh'] ['ThoiSu']
2529 ['ukraine', 'nga', 'poroshenko', 'sky', 'giới', 'putin', 'biên', 'và', 'thống', 'news'] ['TheGioi']
2530 ['csgt', 'xe', 'tá', 'nẵng', 'mừng', 'đà', 'thùng', 'người', 'ôtô', 'phất'] ['ThoiSu']
2531 ['malaysia', 'cổ', 'vé', 'sân', 'việt', 'tuyển', 'nam', 'bóng', 'đá', 'động'] ['TheGioi']
2532 ['startup', 'nghiệp', 'khởi', 'tạo', 'tư', 'đầu', 'các', 'thủ', 'tướng', 'doanh'] ['TheThao']
2533 ['học', 'sĩ', 'dục', 'thụy', 'giáo', 'bằng', 'đại', 'ngữ', 'quản', 'trường'] ['TheGioi']
2534 ['đồn', 'vân', 'tốc', 'long', 'hạ', 'cao', 'thầu', 'gói', 'nối', 'xe'] ['TheThao']
2535 ['sản', 'tài', 'thu', 'nhập', 'nhũng', 'luật', 'cơ', 'quan', 'lý', 'tra'] ['TheThao']
2536 ['hận', 'ủy', 'kỷ', 'xà', 'xã', 'giang', 'hậu', 'đảng', 'luật', 'hệ'] ['ThoiSu']
2537 ['center', 'trade', 'saigon', 'tầng', 'mại', 'toà', 'tòa', 'đàn', 'hầm', 'hoà'] ['TheThao']
2538 ['voi', 'rừng', 'đàn', 'tràm', 'vườn', 'sơn', 'pù', 'con', '

2643 ['vec', 'ổ', 'gà', 'tốc', 'cm', 'thầu', 'cao', 'ngãi', 'bong', 'quảng'] ['TheThao']
2644 ['thiêm', 'ráo', 'hứa', 'quyết', 'giải', 'thủ', 'thật', 'ích', 'sai', 'phong'] ['TheGioi']
2645 ['bush', 'clinton', 'thống', 'cha', 'tổng', 'cựu', 'ngủ', 'giường', 'bill', 'tôi'] ['TheGioi']
2646 ['saraswati', 'thần', 'tượng', 'nữ', 'vĩnh', 'long', 'chiên', 'cổ', 'chùa', 'brahma'] ['ThoiSu']
2647 ['arab', 'saudi', 'khashoggi', 'giết', 'luận', 'tử', 'mỹ', 'báo', 'viện', 'thái'] ['TheGioi']
2648 ['thọ', 'niên', 'xuân', 'thanh', 'bay', 'nhóm', 'viên', 'chụp', 'hung', 'sân'] ['ThoiSu']
2649 ['việt', 'kkk', 'cư', 'nhập', 'seadrift', 'người', 'mỹ', 'dân', 'texas', 'ngư'] ['TheGioi']
2650 ['sắt', 'tốc', 'đường', 'cao', 'án', 'usd', 'dự', 'tư', 'độ', '58'] ['ThoiSu']
2651 ['kho', 'cháy', 'vải', 'chứa', 'vòi', 'rồng', 'tốc', 'cạnh', 'bùng', 'dội'] ['ThoiSu']
2652 ['canada', 'bà', 'mỹ', 'mạnh', 'huawei', 'bắt', 'pháp', 'tòa', 'iran', 'tư'] ['TheGioi']
2653 ['tuyển', 'bổ', 'nhiệm', 'y', 'tra', 'đơn', 'bộ

2730 ['n2', 'đường', 'diễn', 'điểm', 'tuyến', 'khu', 'lộc', 'vốn', 'trâu', 'công'] ['TheThao']
2731 ['ven', 'tàu', 'thủy', 'hạ', 'biển', 'lockheed', 'louis', 'chiến', 'lễ', 'tác'] ['TheGioi']
2732 ['buýt', 'tuyến', 'du', 'city', 'tour', 'hà', 'vé', '000', 'khách', 'xe'] ['TheGioi']
2733 ['cang', 'thành', 'ủy', 'hđnd', 'phạm', 'kiểm', 'ban', 'tất', 'sai', 'biểu'] ['TheGioi']
2734 ['z', 'làng', 'lao', 'điện', 'thôn', 'rừng', 'thuyền', 'bhling', 'bríu', 'đường'] ['ThoiSu']
2735 ['quận', 'chiểu', 'xây', 'tịch', 'phép', 'ông', 'ga', 'uỷ', 'hưng', 'chủ'] ['TheGioi']
2736 ['phiếu', 'tín', 'nhiệm', 'tỉnh', 'hđnd', 'thấp', 'biểu', 'hằng', 'sở', 'đốc'] ['TheThao']
2737 ['manafort', 'mueller', 'tố', 'tội', 'paul', 'trump', 'thỏa', 'cử', 'tòa', 'nga'] ['TheGioi']
2738 ['luân', 'cán', 'chuyển', 'ương', 'chức', 'bộ', 'trung', 'tác', 'ban', 'tổ'] ['TheGioi']
2739 ['park', 'đức', 'cup', 'tuyển', 'hlv', 'đội', 'asian', 'u23', 'quyết', 'anh'] ['TheThao']
2740 ['vec', 'ổ', 'gà', 'tốc', 'cm', 'thầu', 'cao

2838 ['ukraine', 'nga', 'putin', 'tàu', 'chiến', 'kiev', 'biển', 'thủy', 'kerch', 'thủ'] ['TheGioi']
2839 ['lệnh', 'tướng', 'quân', 'tư', 'bổ', 'nhiệm', 'doãn', 'trưởng', 'cương', 'mưu'] ['ThoiSu']
2840 ['bão', 'mm', 'thuận', 'usagi', 'vũng', 'tàu', 'bộ', 'rịa', 'khánh', 'mưa'] ['TheThao']
2841 ['huawei', 'skycom', 'mạnh', 'bà', 'sư', 'gibb', 'carsley', 'mỹ', 'canada', 'martin'] ['TheGioi']
2842 ['bay', 'chuyến', 'tân', 'vòng', 'cánh', 'sơn', 'bangkok', 'hcm', 'xuống', 'nhất'] ['ThoiSu']
2843 ['giáo', 'tát', 'cô', 'học', 'sinh', 'con', 'đống', 'huynh', 'bạn', 'lớp'] ['ThoiSu']
2844 ['cúc', 'vườn', 'hoa', 'mi', 'hạnh', 'hải', 'dương', 'hoạ', 'anh', 'họa'] ['TheGioi']
2845 ['súng', 'thờ', 'xả', 'brazil', 'lễ', 'ap', 'nghi', 'nguyện', 'thật', 'nhà'] ['TheGioi']
2846 ['tốc', 'cầu', 'vec', 'lún', 'mố', 'vd06', 'vd07', 'nứt', 'đầu', 'thầu'] ['TheThao']
2847 ['dâu', 'trại', 'trồng', 'cây', 'trang', 'mộc', 'du', 'quả', 'tây', 'hoa'] ['TheGioi']
2848 ['thuyền', '411', 'sar', 'du', 'nạn', 'tàu',